In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.insert(0,'../')
import StatPol as SP

# Analysis of the statical polarizability for a single study of the HG test set

Discuss the workflow to compute the statical polarizability of a specific study defined by the molecule type, the xc and the psp

Set some global parameters

In [2]:
wf_convergence = 1.0e-6
hgrids = 0.4 # to be set to 0.3 in the production phase
rmult_fine = 9.0

Choose the molecule type, build the associated folder and move there

In [3]:
#%cd ../
molecule = 'CO'
if not os.path.isdir(molecule): os.mkdir(molecule)
%cd $molecule
sys.path.insert(1,'../../')

/home/marco/Data/RICERCA/LINEAR RESPONSE/LR-nb/STATICAL POLARIZABILITY/HG test-set/CO


Set the study type that is the xc functional and the pseudopotential (psp). Build the associated folder and define the path variable

In [4]:
xc = 'pbe' #'lda'
psp = 'hgh-k'
path=xc+'-'+psp
if not os.path.isdir(path): os.mkdir(path)
path

'pbe-hgh-k'

Build a dictionary to collect the results of the analysis

In [5]:
study = {}

In [6]:
# read the posinp from the bigdft database
posinp=Molecules.Molecule(molecule)
print posinp

{'units': 'angstroem', 'positions': [{'C': [0.0, 0.0, 0.0], 'sym': 'C'}, {'sym': 'O', 'O': [0.0, 0.0, 1.1282]}], 'global monopole': 0.0}


In [7]:
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft


Perform a convergence study for the gs. Use the value of the total energy (or the value of the gs dipole) to set the dimension of the box

In [8]:
reload(SP)
rtol=1e-3 #relative tolerance for the gs convergence using the total energy as control quantity

inp = I.Inputfile()
inp.set_hgrid(hgrids)
inp.set_xc(xc.upper())
inp.set_wavefunction_convergence(wf_convergence)

rmult_coarse = [1.0*i for i in range(3,12)]
data = []
code.update_global_options(verbose=False)
for r in rmult_coarse:
    gs_study = D.Dataset(label=molecule+'_GS',run_dir=path,posinp=posinp)
    gs_study.set_postprocessing_function(SP.get_energy)
    #gs_study.set_postprocessing_function(SP-get_dipole)
    inp.set_rmult(coarse=r,fine=rmult_fine)
    idd={'rmult':r}
    gs_study.append_run(id=idd,runner=code,input=inp)
    data.append(gs_study)
    
study['gs_conv'] = SP.seek_convergence(rt=rtol,label='rmult',values=rmult_coarse,data=data)
study

Perform the run with rmult 3.0
Perform the run with rmult 4.0
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Convergence achieved for rmult 4.0


{'gs_conv': {'converged': True,
  'converged_value': 4.0,
  'label': 'rmult',
  'results': {3.0: -21.643196972154243,
   4.0: -21.68076000399936,
   5.0: -21.6835262676615,
   6.0: None,
   7.0: None,
   8.0: None,
   9.0: None,
   10.0: None,
   11.0: None},
  'values': [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]}}

It emerges that the convergence performed with the energy is less demanding that the one done with the dipole!

From this analysis we extract the converged log of the gs to be used as the starting point for the computation of the statical polarizability

In [9]:
conv_val = study['gs_conv']['converged_value']
gslog = 'log-'+data[rmult_coarse.index(conv_val)].names[0]+'.yaml'
print gslog
gs = lf.Logfile(path+os.sep+gslog)

log-rmult:4.0.yaml


Convergence flow for the calculation of alpha: take the rmult of the gs convergence. Perform the field convergence with this value of rmult and finally perform the rmult convergence for alpha using the converged value of the field

In [10]:
rtol=1e-2 #relative tolerance for the alpha convergence

inp = I.Inputfile()
inp.set_hgrid(gs.log['dft']['hgrids'])
inp.set_xc(xc.upper())
inp.set_wavefunction_convergence(gnrm=wf_convergence)
inp.set_rmult(gs.log['dft']['rmult'])
print inp

study['field_conv']=SP.perform_field_convergence(rt=rtol,run_dir=path,input=inp,runner=code,posinp=posinp,ppf=SP.eval_alpha)
print ''

f=study['field_conv']['converged_value']
rmult_list=SP.build_rmult_list(gs)
study['rmult_conv']=SP.perform_rmult_convergence(rt=rtol,run_dir=path,intensity=f,rmult=rmult_list,input=inp,runner=code,posinp=posinp,ppf=SP.eval_alpha)

{'dft': {'ixc': 'PBE', 'rmult': [4.0, 9.0], 'hgrids': 0.4, 'gnrm_cv': 1e-06}}
Perform the run with field_int 0.01
Perform the run with field_int 0.005
Convergence achieved for field_int 0.01

Perform the run with rmult 4.0
Perform the run with rmult 5.0
Convergence for rmult 4.0 failed
Perform the run with rmult 6.0
Convergence for rmult 5.0 failed
Perform the run with rmult 7.0
Convergence achieved for rmult 6.0


In [11]:
r_conv = study['rmult_conv']['converged_value']
alpha_final = study['rmult_conv']['results'][r_conv]
print alpha_final
print SP.eval_alpha_avg(alpha_final)

[[ 1.2511570e+01 -2.8314000e-05 -4.3799000e-05]
 [-2.8314000e-05  1.2511570e+01 -4.3799000e-05]
 [-4.6000000e-05 -4.6000000e-05  1.5899891e+01]]
13.641010333333332


In [12]:
study

{'field_conv': {'converged': True,
  'converged_value': 0.01,
  'label': 'field_int',
  'results': {0.001: None,
   0.005: matrix([[ 1.1583312e+01, -4.1900000e-04,  9.4440000e-04],
           [-4.1900000e-04,  1.1583312e+01,  9.4440000e-04],
           [ 9.7000000e-04,  9.7000000e-04,  1.4620368e+01]]),
   0.01: matrix([[ 1.15910800e+01, -4.17000000e-04,  9.21650000e-04],
           [-4.17000000e-04,  1.15910800e+01,  9.21650000e-04],
           [ 9.25000000e-04,  9.25000000e-04,  1.46287645e+01]])},
  'values': [0.01, 0.005, 0.001]},
 'gs_conv': {'converged': True,
  'converged_value': 4.0,
  'label': 'rmult',
  'results': {3.0: -21.643196972154243,
   4.0: -21.68076000399936,
   5.0: -21.6835262676615,
   6.0: None,
   7.0: None,
   8.0: None,
   9.0: None,
   10.0: None,
   11.0: None},
  'values': [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]},
 'rmult_conv': {'converged': True,
  'converged_value': 6.0,
  'label': 'rmult',
  'results': {4.0: matrix([[ 1.15910800e+01, -4.17000000

This workflow will be translated into a single method to be used at higher level when one is dealing with several studies...